In [1]:
#=======================================================================================
# sentencepiece 사전 모델 처음부터 만드는 예제
# => 말뭉치를 가지고, sp vocab을 만듬(학습)
#
# [인자]
# -input_file : 입력할 말뭉치 데이터 경로 지정
# -vocab_size : BPE의 단어수를 얼마로 할 것인가 이다. 너무 적으면 한 글자 단위로 쪼개지는 경향이 있고, 너무 많으면 쓸데없는 단어들이 만들어진다. 
#               주로 3,2000이 가장 좋다고 알려져 있다.
# -model_name : 저장할 이름이다. 학습하고 나면 .model, .vocab 2개의 파일이 만들어진다.
# -model_type : bpe, unigram 등이 있는데 두가지를 모두 사용해 보고 성능이 좋은 거로 고르도록 하자.
# -character_coverage : 모든 단어를 커버할것인가, 너무 희귀한 단어는 뺄 것인가 이다. 학습 코퍼스가 대용량이라면 보통 0.9995로 사용하면 된다. 그런데 코퍼스가 작다면 1.0으로 지정하자. 그럼 [UNK]가 없다.
# -user_defined_symbols : BPE로 생성된 단어 외 알고리즘에서 사용할 특수문자들을 지정한다. BERT 기반 요약, 번역모델 개발 시 BOS, EOS, SEP 등 추가 토큰들이 필요하다. 
#                         그러므로 user_defined_symbols을 이용해 Dummy token(UNK, unused, BOS, EOS 등등)을 추가해준다. 
#                         HTTP나 network traffic에 BPE를 적용해야 할 경우가 있다. 
#                         이때 user_defined_symbols 인자에 'http, GET, POST, User-Agent 등 HTTP 데이터에서 자주 나오는 Token들을 지정하기도 했다.
# -vocab_size에 비례하여 학습시간이 결정된다. 필자는 32,000으로 지정하여 노트북에서 대략 43sec가 소요됐다.
# -input_sentence_size : 입력문장 size가 너무 크면 OOM 발생함. 따라서 적당한 사이즈를 지정함(보통 7G=700000 지정함)
# -shuffle_input_sentence : 입력문장을 섞는다.
#
# sentencepiecr 설치
#!pip install sentencepiece
#=======================================================================================
import sentencepiece as spm
import os
import time
from time import localtime

In [2]:
#==================================================================================
# parameter 설정 
#==================================================================================
input_file = '../../data11/ai_hub/tl1/tl1-1줄-mecab.txt'

# 모델 root 경로 지정
model_root = '../../data11/ai_hub/vocab/tl1-1줄-mecab-30000-sp-unigram-22000000/'
if not os.path.isdir(model_root):
    os.mkdir(model_root)
    
vocab_size=30000

# 모델명과, 풀경로 지정    
model_name = 'spm_{}_token'.format(vocab_size)
model_fullpath = os.path.join(model_root, model_name)
print(model_fullpath)

model_type = 'unigram' #'bpe' # model_type : 사용할 모델 (unigram(default), bpe, char, word)

character_coverage = 0.9995#1.0 # 0.9995 # 모든 단어 커버할지(1.0 혹은 0.9995로 지정)
user_defined_symbols = '[PAD],[UNK],[CLS],[SEP],[MASK],[BOS],[EOS]'

input_sentence_size=22000000 # 입력 문장 샘플링 크기 제한(*OOM 발생하면, size를 더 줄인다)
shuffle_input_sentence=True # 문장을 섞는다.
num_threads=35

../../data11/ai_hub/vocab/tl1-1줄-mecab-30000-sp-unigram-22000000/spm_30000_token


In [3]:
# 훈련 파라메터 설정
input_argument = '--input={0}\
 --model_prefix={1}\
 --vocab_size={2}\
 --user_defined_symbols={3}\
 --model_type={4}\
 --input_sentence_size={5}\
 --shuffle_input_sentence={6}\
 --num_threads={7}\
 --character_coverage={8}'\
.format(input_file, model_fullpath, vocab_size, user_defined_symbols, model_type, 
        input_sentence_size, shuffle_input_sentence, num_threads, character_coverage)

print(input_argument)

# 시작시간
start = time.time()
tm = localtime(start)
print(f'=== 시작시간: {tm.tm_year}-{tm.tm_mon}-{tm.tm_mday} {tm.tm_hour}:{tm.tm_min}:{tm.tm_sec} ===')

--input=../../data11/ai_hub/tl1/tl1-1줄-mecab.txt --model_prefix=../../data11/ai_hub/vocab/tl1-1줄-mecab-30000-sp-unigram-22000000/spm_30000_token --vocab_size=30000 --user_defined_symbols=[PAD],[UNK],[CLS],[SEP],[MASK],[BOS],[EOS] --model_type=unigram --input_sentence_size=22000000 --shuffle_input_sentence=True --num_threads=35 --character_coverage=0.9995
=== 시작시간: 2022-11-25 14:8:7 ===


In [4]:
# 훈련 시작
# => 훈련이 완료되고 나면 해당경로에 xxx.model, xxx.vocab 2개의 파일이 생성된다. (오래 걸림)
spm.SentencePieceTrainer.Train(input_argument)

print(f'=== 처리시간: {time.time() - start:.3f} 초 ===')

=== 처리시간: 2414.223 초 ===


sentencepiece_trainer.cc(177) LOG(INFO) Running command: --input=../../data11/ai_hub/tl1/tl1-1줄-mecab.txt --model_prefix=../../data11/ai_hub/vocab/tl1-1줄-mecab-30000-sp-unigram-22000000/spm_30000_token --vocab_size=30000 --user_defined_symbols=[PAD],[UNK],[CLS],[SEP],[MASK],[BOS],[EOS] --model_type=unigram --input_sentence_size=22000000 --shuffle_input_sentence=True --num_threads=35 --character_coverage=0.9995
sentencepiece_trainer.cc(77) LOG(INFO) Starts training with : 
trainer_spec {
  input: ../../data11/ai_hub/tl1/tl1-1줄-mecab.txt
  input_format: 
  model_prefix: ../../data11/ai_hub/vocab/tl1-1줄-mecab-30000-sp-unigram-22000000/spm_30000_token
  model_type: UNIGRAM
  vocab_size: 30000
  self_test_sample_size: 0
  character_coverage: 0.9995
  input_sentence_size: 22000000
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 35
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script

In [5]:
# 검증 시작
sp = spm.SentencePieceProcessor()
sp.Load('{}.model'.format(model_fullpath))

tokens = sp.encode_as_pieces('나는 아침에 배를 먹었다')
ids = sp.encode_as_ids('나는 아침에 배를 먹었다')

print(ids)
print(tokens)

tokens = sp.decode_pieces(tokens)
ids = sp.decode_ids(ids)

print(ids)
print(tokens)

[104, 3164, 1610, 3518, 485, 2696, 980, 4898, 670]
['▁나', '는', '▁아침', '에', '▁배', '를', '▁먹', '었', '다']
나는 아침에 배를 먹었다
나는 아침에 배를 먹었다


obj=7.41986 num_tokens=1050521 num_tokens/piece=15.5882
unigram_model_trainer.cc(505) LOG(INFO) EM sub_iter=1 size=67392 obj=7.41868 num_tokens=1050515 num_tokens/piece=15.5881
unigram_model_trainer.cc(505) LOG(INFO) EM sub_iter=0 size=50544 obj=7.43028 num_tokens=1115248 num_tokens/piece=22.0649
unigram_model_trainer.cc(505) LOG(INFO) EM sub_iter=1 size=50544 obj=7.42758 num_tokens=1115244 num_tokens/piece=22.0648
unigram_model_trainer.cc(505) LOG(INFO) EM sub_iter=0 size=37908 obj=7.44841 num_tokens=1185556 num_tokens/piece=31.2746
unigram_model_trainer.cc(505) LOG(INFO) EM sub_iter=1 size=37908 obj=7.44385 num_tokens=1185564 num_tokens/piece=31.2748
unigram_model_trainer.cc(505) LOG(INFO) EM sub_iter=0 size=33000 obj=7.45929 num_tokens=1218025 num_tokens/piece=36.9098
unigram_model_trainer.cc(505) LOG(INFO) EM sub_iter=1 size=33000 obj=7.45623 num_tokens=1218026 num_tokens/piece=36.9099
trainer_interface.cc(615) LOG(INFO) Saving model: ../../data11/ai_hub/vocab/tl1-1줄-mecab-30000-sp